# Qwen/Qwen3-VL-8B-Instruct 제로샷 테스트
- bf16/float16
- 학습 없이 바로 추론
- 테스트데이터 3887개 전부 추론 => 캐글 제출용

### [ 실행 시간 (colab A100 환경) ]
- [추론] Total: 11.01 min | per-sample: 0.166 s

### [ 캐글 스코어 : 94804 ]


### 다음에 해볼 것



In [ ]:
# ✅ Colab 환경 준비: CUDA 12.1용 PyTorch + 필수 라이브러리 설치 후 자동 재시작
import os, time
print("필수 패키지 설치 중... ⏳")

# Hugging Face/PEFT/bitsandbytes 등
!pip -q install "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.43.1" pillow pandas --upgrade
# Transformer 최신 버전 설치 (소스에서 설치 권장)
!pip install "git+https://github.com/huggingface/transformers"


# ⚠️ 핵심: CUDA 12.1용 PyTorch/torchvision 설치 (CPU 빌드 방지)
!pip -q install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu121

print("\n✅ 설치 완료. 런타임을 자동으로 재시작합니다...")
time.sleep(2)
os.kill(os.getpid(), 9)  # 자동 재시작



필수 패키지 설치 중... ⏳
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-cs8l_7j3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-cs8l_7j3
  Resolved https://github.com/huggingface/transformers to commit e2e8dbed13c6a8455fd85c15c9fa91c99d609010
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.0/502.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11348359 sha256=193ef30f68ccb9cd34434ad8a3f5aeadc90c42895796769f6ad6eaa0461942ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-9l5wxnox/wheels/49/a7/50/c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers
  Attempting uninstall: huggingface-hub
   

In [1]:

# ✅ GPU 사용 가능 여부/드라이버 확인
import time, torch
t0 = time.perf_counter()

# NVIDIA 드라이버/메모리 상태
!nvidia-smi

print("GPU 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch CUDA 버전:", torch.version.cuda)
if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))

# ⏱️ 측정
if torch.cuda.is_available(): torch.cuda.synchronize()
print(f"[체크 소요시간] {(time.perf_counter()-t0):.3f}s")


Sat Oct 25 10:48:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
# ✅ Google Drive 마운트 및 데이터 압축 해제 (한글 주석)
from google.colab import drive
import os, time

drive.mount('/content/drive')

zip_path = "/content/drive/My Drive/aichallenge/data.zip"
unzip_target = "/content/"

need_unzip = (not os.path.isdir('/content/test')) or (len(os.listdir('/content/test')) == 0)
if need_unzip:
    t0 = time.perf_counter()
    print("데이터 압축 해제 중... ⏳")
    !unzip -q "/content/drive/My Drive/aichallenge/data.zip" -d "/content/"
    print("압축 해제 완료.")
    print(f"[압축 해제 소요시간] {(time.perf_counter()-t0):.2f}s")
else:
    print("데이터가 이미 준비되어 있어 압축 해제를 건너뜁니다.")

assert os.path.exists('/content/test.csv') and os.path.exists('/content/test'), "데이터 경로 확인 필요"
print("확인 완료: /content/test.csv, /content/test/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
데이터가 이미 준비되어 있어 압축 해제를 건너뜁니다.
확인 완료: /content/test.csv, /content/test/


### 라이브러리 임포트 & 공통 유틸

In [3]:
# 📦 임포트
import time, re, random
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from transformers import AutoProcessor
from transformers import Qwen3VLForConditionalGeneration  # 공식 Quickstart 권장 클래스

# 🔧 재현성(선택)
SEED = 42
random.seed(SEED); torch.manual_seed(SEED);
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 🧠 시스템 지시문(정답 한 글자만)
SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)

# ✅ 모델 응답에서 선지(a/b/c/d)만 추출
def extract_choice(text: str) -> str:
    t = text.strip().lower()
    # 마지막 줄/토큰에서 우선 탐색
    lines = [l.strip() for l in t.splitlines() if l.strip()]
    last = lines[-1] if lines else t
    if last in ["a","b","c","d"]:
        return last
    for tok in re.findall(r"[a-d]", last):
        return tok
    return "a"  # 안전장치


###데이터 로드

In [4]:
# 📥 CSV 로드 (대회 포맷)
test_df  = pd.read_csv("/content/test.csv")   # id, path, question, a,b,c,d
sample   = pd.read_csv("/content/sample_submission.csv")  # id, answer

print("test size:", len(test_df))
test_df.head(2)

test size: 3887


,id,path,question,a,b,c,d
0,test_0001,test/test_0001.jpg,이 사진에 보이는 탑은 무엇인가요?,63빌딩,남산타워,롯데타워,한강대교
1,test_0002,test/test_0002.jpg,이 사진에서 볼 수 있는 나무의 잎 색깔은 무엇인가요?,초록색,노란색,파란색,빨간색


### 모델/프로세서 로드

In [5]:
# 🧩 모델/프로세서 로드 (공식 예제 형식)
MODEL_ID = "Qwen/Qwen3-VL-8B-Instruct"

# dtype="auto", device_map="auto" → A100에서 자동 bf16/float16로 배치됨
# (flash_attention_2는 선택. Colab에서 빌드 이슈가 자주 있어 기본 비활성 권장)
model = Qwen3VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(MODEL_ID)
print("Loaded:", MODEL_ID)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

Loaded: Qwen/Qwen3-VL-8B-Instruct


### ✋pilot 50개 정확도 테스트

In [6]:
# ✅ pilot_300.csv 정확도 테스트 (앞 50개)
import pandas as pd
import numpy as np
import time
from PIL import Image
from tqdm import tqdm

# 📥 파일 경로
PILOT_PATH = "/content/drive/My Drive/aichallenge/pilot_300.csv"

# ⚙️ 상수
N_SAMPLES = 50   # 상위 50개만 테스트
INTERVAL = 10    # 10개 단위로 결과 출력

# 📄 데이터 로드
pilot_df = pd.read_csv(PILOT_PATH)
pilot_df = pilot_df.head(N_SAMPLES)
print(f"Loaded pilot samples: {len(pilot_df)}")

# ⏱️ 전체 시간 측정 시작
t0_total = time.time()
preds, trues = [], []
per_sample_times = []

# 🔥 추론 루프
model.eval()
with torch.no_grad():
    for i in tqdm(range(len(pilot_df)), desc="Pilot Test", unit="sample"):
        row = pilot_df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")

        # 사용자 프롬프트 구성
        user_text = (
            f"{row['question']}\n"
            f"(a) {row['a']}\n(b) {row['b']}\n(c) {row['c']}\n(d) {row['d']}\n\n"
            "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
        )

        messages = [
            {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
            {"role": "user",   "content": [
                {"type": "image", "image": img},
                {"type": "text",  "text": user_text},
            ]},
        ]

        # ⏱️ per-sample 시간 측정 시작
        t0 = time.time()

        # 입력 준비
        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # 🔮 생성
        gen_ids = model.generate(
            **inputs,
            max_new_tokens=2,
            do_sample=False,
            temperature=0.0,
            eos_token_id=processor.tokenizer.eos_token_id,
        )

        trimmed = [out[len(inp):] for inp, out in zip(inputs["input_ids"], gen_ids)]
        out_text = processor.batch_decode(trimmed, skip_special_tokens=True)[0]

        pred = extract_choice(out_text)
        true = str(row["answer"]).strip().lower()

        preds.append(pred)
        trues.append(true)
        per_sample_times.append(time.time() - t0)

        # 📈 10개 단위 중간 출력
        if (i + 1) % INTERVAL == 0:
            subset = preds[:i+1]
            subset_true = trues[:i+1]
            acc = np.mean(np.array(subset) == np.array(subset_true))
            wrong = np.sum(np.array(subset) != np.array(subset_true))
            print(f"👉 {i+1}개 중간 결과: 정확도 {acc*100:.2f}% | 오답 {wrong}개")

# ✅ 전체 결과
total_acc = np.mean(np.array(preds) == np.array(trues))
total_wrong = np.sum(np.array(preds) != np.array(trues))
print(f"\n✅ 전체 50개 결과: 정확도 {total_acc*100:.2f}% | 오답 {total_wrong}개")
print(f"⏱️ 총 소요 시간: {time.time()-t0_total:.2f}초 | 1개당 평균 {np.mean(per_sample_times):.3f}초")


Loaded pilot samples: 50


Pilot Test:  22%|██▏       | 11/50 [00:04<00:07,  5.14sample/s]

👉 10개 중간 결과: 정확도 100.00% | 오답 0개


Pilot Test:  42%|████▏     | 21/50 [00:06<00:04,  5.85sample/s]

👉 20개 중간 결과: 정확도 100.00% | 오답 0개


...

👉 30개 중간 결과: 정확도 96.67% | 오답 1개


Pilot Test:  82%|████████▏ | 41/50 [00:09<00:01,  6.04sample/s]

👉 40개 중간 결과: 정확도 92.50% | 오답 3개


Pilot Test: 100%|██████████| 50/50 [00:11<00:00,  4.39sample/s]

👉 50개 중간 결과: 정확도 94.00% | 오답 3개

✅ 전체 50개 결과: 정확도 94.00% | 오답 3개
⏱️ 총 소요 시간: 11.39초 | 1개당 평균 0.223초


### 제로샷 추론 루프

In [7]:
# ⏱️ 전체/개별 시간 측정 + 메모리 절약 팁
model.eval()
total_t0 = time.time()
per_sample = []
preds = []

with torch.no_grad():
    for i in tqdm(range(len(test_df)), desc="Inference", unit="sample"):
        row = test_df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")

        # ✍️ 프롬프트 (문항 + 4지 선지)
        user_text = (
            f"{row['question']}\n"
            f"(a) {row['a']}\n(b) {row['b']}\n(c) {row['c']}\n(d) {row['d']}\n\n"
            "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
        )

        # 💬 메시지(시스템+유저) → 공식 권장 방식(apply_chat_template)
        messages = [
            {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
            {"role": "user",   "content": [
                {"type": "image", "image": img},
                {"type": "text",  "text": user_text},
            ]},
        ]

        # ⏱️ per-sample 시작
        t0 = time.time()

        # 🧴 텐서화: tokenize=True + add_generation_prompt=True + return_tensors="pt"
        # 공식 카드에 맞춰, 이미지 포함 메시지를 processor가 한 번에 처리
        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # 🧪 생성: 단답(문자 1개)을 강제하기 위해 max_new_tokens=2, 샘플링 비활성화
        gen_ids = model.generate(
            **inputs,
            max_new_tokens=2,
            do_sample=False,               # greedy
            temperature=0.0,
            eos_token_id=processor.tokenizer.eos_token_id,
        )

        # 🔎 입력 길이만큼 잘라서 "새로 생성된 토큰"만 decode (공식 Quickstart 패턴)
        trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs["input_ids"], gen_ids)]
        out_text = processor.batch_decode(trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

        # ✅ 한 글자 레이블 추출
        preds.append(extract_choice(out_text))

        # ⏱️ per-sample 종료
        per_sample.append(time.time() - t0)

total_sec = time.time() - total_t0
print(f"⏱️ Total: {total_sec/60:.2f} min | per-sample: {sum(per_sample)/len(per_sample):.3f} s")


Inference: 100%|██████████| 3887/3887 [11:00<00:00,  5.89sample/s]

⏱️ Total: 11.01 min | per-sample: 0.166 s


### 제출 파일 생성

In [8]:
# 📄 sample_submission 포맷에 맞춰 저장
submission = pd.DataFrame({
    "id": test_df["id"],
    "answer": preds
})
save_path = "/content/submission.csv"
submission.to_csv(save_path, index=False)
print("✅ Saved:", save_path)
submission.head()


✅ Saved: /content/submission.csv


,id,answer
0,test_0001,b
1,test_0002,b
2,test_0003,b
3,test_0004,c
4,test_0005,c
